In [1]:
from pynq import Overlay

overlay = Overlay("./lab_2.bit")

In [2]:
overlay?

In [3]:
top_func = overlay.lab_2_0

In [4]:
top_func.register_map

RegisterMap {
  A_V = Register(A_V=0),
  B_V = Register(B_V=0),
  C_V = Register(C_V=0),
  Y_V = Register(Y_V=0),
  Y_V_ctrl = Register(Y_V_ap_vld=1, RESERVED=0)
}

In [5]:
# test values
val_A = 12732.636
val_B = 3434.21
val_C = 38.1223

In [6]:
"""
f is the input floating point number 
e is the number of fractional bits in the ap_fixed<W,I> format
"""
def to_fixed(f,e):
    a = f* (2**e)
    b = int(round(a))
    if a < 0:
        # next three lines turns b into it's 2's complement.
        b = abs(b)
        b = ~b
        b = b + 1
    return b

fix_val_A = to_fixed(val_A, 16)
fix_val_B = to_fixed(val_B, 16)
fix_val_C = to_fixed(val_C, 16)

In [7]:
print(f"fix_val_A: {fix_val_A}")
print(f"fix_val_B: {fix_val_B}")
print(f"fix_val_C: {fix_val_C}")

fix_val_A: 834446033
fix_val_B: 225064387
fix_val_C: 2498383


In [8]:
top_func.register_map.A_V = fix_val_A
top_func.register_map.B_V = fix_val_B
top_func.register_map.C_V = fix_val_C

In [9]:
top_func.register_map

RegisterMap {
  A_V = Register(A_V=834446033),
  B_V = Register(B_V=225064387),
  C_V = Register(C_V=2498383),
  Y_V = Register(Y_V=1002857),
  Y_V_ctrl = Register(Y_V_ap_vld=1, RESERVED=0)
}

In [10]:
print(top_func.register_map.Y_V)

0xf4d69


In [11]:
val = top_func.read(0x28)
print(f"{val} {type(val)}")

1002857 <class 'int'>


In [12]:
"""
x is the input fixed number which is of integer datatype
e is the number of fractional bits in the ap_fixed<W,I> format
"""
def to_float(x,e):
    c = abs(x)
    sign = 1 
    if x < 0:
        # convert back from two's complement
        c = x - 1 
        c = ~c
        sign = -1
    f = (1.0 * c) / (2 ** e)
    f = f * sign
    return f

float_val = to_float(val, 16)
print(float_val)

15.302383422851562


In [13]:
# ground truth
import math
print(math.log(val_A/val_C) + math.sqrt(val_B/val_C))

15.302384468324684
